In [ ]:
import os
import shutil
import tempfile
import time
import urllib.request
import zipfile

version = "1.10"
repo = "HarukiShimodaira/4-4-4-Cube-Analysis-Simulator"
repo_name = repo.split("/", 1)[1]

base_dir = "/content" if os.path.isdir("/content") else os.path.join(tempfile.gettempdir(), "cube_analysis_simulator")
os.makedirs(base_dir, exist_ok=True)

zip_filename = f"{repo_name}-{version}.zip"
zip_path = os.path.join(base_dir, zip_filename)
url = f"https://codeload.github.com/{repo}/zip/refs/tags/{version}"

app_folder = f"{repo_name}-{version}"
app_dir = os.path.join(base_dir, app_folder)

print(f"Downloading ZIP file from GitHub (version {version})...")

if os.path.exists(zip_path):
    os.remove(zip_path)

if os.path.isdir(app_dir):
    shutil.rmtree(app_dir)

urllib.request.urlretrieve(url, zip_path)
print(f"Download complete: {zip_filename}")

print("Extracting...")
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(base_dir)

if not os.path.isfile(os.path.join(app_dir, "run_web_gui.py")):
    candidates = []
    for name in os.listdir(base_dir):
        path = os.path.join(base_dir, name)
        if os.path.isdir(path) and os.path.isfile(os.path.join(path, "run_web_gui.py")):
            candidates.append(path)
    if candidates:
        app_dir = sorted(candidates)[-1]
    else:
        raise FileNotFoundError("run_web_gui.py not found after extraction")

os.chdir(app_dir)
print(f"Current directory: {os.getcwd()}")
print("run_web_gui.py found" if os.path.exists("run_web_gui.py") else "run_web_gui.py not found")

time.sleep(1)
print("Installing packages...")
!pip -q install -r requirements.txt pyngrok 'qrcode[pil]'
print("Setup complete!")

APP_DIR = app_dir

Download complete: 4-4-4-Cube-Analysis-Simulator-1.10.zip
Extracting...
Current directory: /Users/haruki/Documents/Rubiks_cube_simulator_Release_Beta_v4/4-4-4-Cube-Analysis-Simulator-1.10
run_web_gui.py found
Installing packages...
Setup complete!


In [ ]:
from pyngrok import ngrok
import subprocess
import tempfile
import time
import qrcode
from IPython.display import display, Image as IPImage
import io
import os
import sys
import socket
from getpass import getpass

version = "1.10"

def ensure_app_dir():
    global APP_DIR
    if "APP_DIR" in globals() and APP_DIR and os.path.isdir(APP_DIR):
        os.chdir(APP_DIR)
        return

    local_base = os.path.join(tempfile.gettempdir(), "cube_analysis_simulator")
    bases = [os.getcwd(), "/content", local_base]

    for base in bases:
        if not os.path.isdir(base):
            continue
        if os.path.isfile(os.path.join(base, "run_web_gui.py")):
            APP_DIR = base
            os.chdir(APP_DIR)
            return
        for name in os.listdir(base):
            path = os.path.join(base, name)
            if os.path.isfile(os.path.join(path, "run_web_gui.py")):
                APP_DIR = path
                os.chdir(APP_DIR)
                return

    raise FileNotFoundError("run_web_gui.py not found")

def wait_for_server(port=8888, timeout=60):
    print("Preparing server...")
    deadline = time.time() + timeout
    i = 0
    while time.time() < deadline:
        i += 1
        time.sleep(1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            result = sock.connect_ex(("127.0.0.1", port))
        finally:
            sock.close()
        if result == 0:
            print(f"Server started confirmation ({i} seconds later)")
            return True
        print(f"Waiting... {i} seconds")
    return False

def generate_qr_code(url):
    print("Generating QR code...")
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(url)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)

    display(IPImage(buf.read()))

def get_ngrok_authtoken():
    token = os.environ.get("NGROK_AUTHTOKEN")
    if token and token.strip():
        return token.strip()

    try:
        from google.colab import userdata
        token = userdata.get("NGROK_AUTHTOKEN") or userdata.get("ngrok_authtoken") or userdata.get("NGROK_TOKEN")
        if token and str(token).strip():
            return str(token).strip()
    except Exception:
        pass

    print("Enter your ngrok authtoken")
    print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
    token = getpass("Authtoken: ")
    if token and token.strip():
        return token.strip()

    raise RuntimeError("NGROK_AUTHTOKEN is missing")

ensure_app_dir()
print(f"Current directory: {os.getcwd()}")

authtoken = get_ngrok_authtoken()
os.environ["NGROK_AUTHTOKEN"] = authtoken

try:
    ngrok.kill()
except Exception:
    pass

ngrok.set_auth_token(authtoken)

print("Starting Flask server...")
subprocess.run(["pkill", "-f", "run_web_gui.py"], check=False)
time.sleep(1)

process = subprocess.Popen([sys.executable, "run_web_gui.py"])

if not wait_for_server():
    process.terminate()
    raise RuntimeError("Server failed to start")

listener = ngrok.connect(addr=8888, proto="http")
url_str = listener.public_url

print("\n" + "=" * 60)
print(f"4×4×4 Cube Analysis Simulator - Web GUI v{version}")
print("=" * 60)
print("Launch completed")
print("=" * 60)
print("\nAccess the following URL on your device:")
print(url_str)
print("\nImportant: Stopping this cell will also stop the server")
print("=" * 60)

generate_qr_code(url_str)

try:
    while True:
        time.sleep(10)
        if process.poll() is not None:
            print("\nServer stopped")
            break
except KeyboardInterrupt:
    print("\nStopped")
finally:
    try:
        process.terminate()
    except Exception:
        pass
    try:
        ngrok.kill()
    except Exception:
        pass